<a href="https://colab.research.google.com/github/venkatkp50/Espnet/blob/main/Espnet2_course_2023_speaker_ass3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/ftshijt/notebook/blob/master/ESPnet2/Course/CMU_SpeechProcessing_Spring2023/assignment3_spk.ipynb

In [1]:

!git clone --depth 5 -b 2023spring_speaker_recognition https://github.com/espnet/espnet

%cd /content/espnet/tools
!./setup_anaconda.sh anaconda espnet 3.9

# # It may take 12 minutes
%cd /content/espnet/tools
!make TH_VERSION=1.12.1 CUDA_VERSION=11.6

!. ./activate_python.sh && installers/install_speechbrain.sh
!. ./activate_python.sh && installers/install_rawnet.sh
!. ./activate_python.sh && pip install ipykernel

Streaming output truncated to the last 5000 lines.





python-3.9.21        | 25.1 MB   | :  15% 0.15246078732766938/1 [00:00<00:01,  1.75s/it]




xz-5.6.4             | 567 KB    | : 100% 1.0/1 [00:00<00:00,  8.23s/it]                 

setuptools-75.8.0    | 1.6 MB    | : 100% 1.0/1 [00:00<00:00,  5.10it/s]               





libcurl-8.12.0       | 468 KB    | : 100% 1.0/1 [00:00<00:00,  8.19s/it]                






zstandard-0.23.0     | 431 KB    | :   4% 0.03710674303625744/1 [00:00<00:08,  8.97s/it]







boltons-24.1.0       | 422 KB    | :   4% 0.037925925925925925/1 [00:00<00:08,  9.25s/it]
pip-25.0             | 2.3 MB    | : 100% 1.0/1 [00:00<00:00,  4.38it/s]               








brotli-python-1.0.9  | 357 KB    | :   4% 0.04486235165906365/1 [00:00<00:07,  8.14s/it]







boltons-24.1.0       | 422 KB    | : 100% 1.0/1 [00:00<00:00,  9.25s/it]                 






python-3.9.21        | 25.1 MB   | :  21% 0.21033365761939693/1 [00:00<00:01,  1.76s/it]











In [2]:
%cd /content/espnet/egs2/mini_librispeech/sid1
!ls -l


/content/espnet/egs2/mini_librispeech/sid1
total 28
lrwxrwxrwx 1 root root   26 Feb 14 05:18 asr.sh -> ../../TEMPLATE/asr1/asr.sh
-rw-r--r-- 1 root root 4231 Feb 14 05:18 cmd.sh
drwxr-xr-x 2 root root 4096 Feb 14 05:18 conf
lrwxrwxrwx 1 root root   25 Feb 14 05:18 db.sh -> ../../TEMPLATE/asr1/db.sh
drwxr-xr-x 2 root root 4096 Feb 14 05:18 local
lrwxrwxrwx 1 root root   27 Feb 14 05:18 path.sh -> ../../TEMPLATE/asr1/path.sh
lrwxrwxrwx 1 root root   29 Feb 14 05:18 pyscripts -> ../../TEMPLATE/asr1/pyscripts
-rwxr-xr-x 1 root root  724 Feb 14 05:18 run.sh
-rwxr-xr-x 1 root root  816 Feb 14 05:18 run_xvector.sh
-rwxr-xr-x 1 root root  828 Feb 14 05:18 run_xvector_speechbrain.sh
lrwxrwxrwx 1 root root   27 Feb 14 05:18 scripts -> ../../TEMPLATE/asr1/scripts
lrwxrwxrwx 1 root root   37 Feb 14 05:18 steps -> ../../../tools/kaldi/egs/wsj/s5/steps
lrwxrwxrwx 1 root root   37 Feb 14 05:18 utils -> ../../../tools/kaldi/egs/wsj/s5/utils


In [3]:
!./run.sh --stage 1 --stop_stage 1

2025-02-14T05:36:03 (asr.sh:267:main) ./asr.sh --train_set train --valid_set dev --test_sets test --ngpu 1 --asr_config conf/train_sid_transformer.yaml --use_lm false --inference_config conf/decode_sid.yaml --inference_nj 5 --token_type word --local_data_opts --stage 0 --stage 1 --stop_stage 1
/content/espnet/tools/anaconda/envs/espnet/etc/conda/activate.d/libblas_mkl_activate.sh: line 1: MKL_INTERFACE_LAYER: unbound variable


In [5]:
!./run.sh --stage 2 --stop_stage 5

2025-02-14T05:36:45 (asr.sh:267:main) ./asr.sh --train_set train --valid_set dev --test_sets test --ngpu 1 --asr_config conf/train_sid_transformer.yaml --use_lm false --inference_config conf/decode_sid.yaml --inference_nj 5 --token_type word --local_data_opts --stage 0 --stage 2 --stop_stage 5
/content/espnet/tools/anaconda/envs/espnet/etc/conda/activate.d/libblas_mkl_activate.sh: line 1: MKL_INTERFACE_LAYER: unbound variable


In [6]:


!cat ./local/extract_xvector.sh


!./local/extract_xvector.sh



#!/bin/bash
  
# Copyright 2023 Carnegie Mellon University (Jiatong Shi)
# Apache 2.0  (http://www.apache.org/licenses/LICENSE-2.0)

. ./path.sh || exit 1;
. ./cmd.sh || exit 1;
. ./db.sh || exit 1;

mkdir -p dump/extracted

if [ ! -f "model.pt" ]; then
    wget https://huggingface.co/jungjee/RawNet3/resolve/main/model.pt
fi

for dset in train dev test; do
    datadir=dump/raw/${dset}
    outdir=dump/extracted/${dset}
    cp -r ${datadir} ${outdir}
    python3 pyscripts/utils/extract_xvectors.py --toolkit rawnet \
        --pretrained_model model.pt --draw_tsne true --tsne_n_spk 10 \
       ${datadir} ${outdir}

    cp ${outdir}/xvector.scp ${outdir}/feats.scp
    echo "256" > ${outdir}/feats_dim
    echo "xvector" > ${outdir}/feats_type
done

--2025-02-14 05:37:11--  https://huggingface.co/jungjee/RawNet3/resolve/main/model.pt
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.4, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443.

In [7]:
from IPython.display import Image, display
display(Image('dump/extracted/train/tsne.png'))

FileNotFoundError: No such file or directory: 'dump/extracted/train/tsne.png'

FileNotFoundError: No such file or directory: 'dump/extracted/train/tsne.png'

<IPython.core.display.Image object>